Шанарова Надежда, 451 группа








In [ ]:
import numpy as np
import copy

n = 3
def_system = np.array([[3.278164, 1.046583, -1.378574, -0.527466], 
                       [1.046583, 2.975937, 0.934251, 2.526877], 
                       [-1.378574, 0.934251, 4.836173, 5.165441]])

spec_rad = 0.620403

simple_system = np.array([[1., 1., 1., 3.], 
                          [2., 1., 1., 2.], 
                          [2., 2., 1., 1.]])

**Получение решения методом Гаусса**

In [ ]:
#выполнение прямого хода
def forward(system, n, expansion): 
  for k in range(0, n):
    tmp = system[k, k]
    for j in range(k, n + expansion):
      system[k, j] = system[k, j] / tmp
    for i in range(k+1, n):
      tmp = system[i, k]
      for j in range(k, n + expansion):
        system[i, j] = system[i, j] - system[k, j] * tmp

#выполнение обртаного хода
def get_solution(system, n):
  x = np.zeros(n)
  for i in range(n-1, -1, -1):
    sum = 0;
    for j in range(n-1, i, -1):
      sum += system[i, j] * x[j]
    x[i] = system[i, n] - sum
  return x

#получение вектора b из расширенной системы
def get_b(system, n):
  b = np.zeros(n)
  for i in range(0, n):
    b[i] = system[i, n]
  return b

#получение матрицы A из расширенной системы
def get_a(system, n):
  A = np.zeros((n, n))
  for i in range(0, n):
    for j in range(0, n):
      A[i, j] = system[i, j]
  return A

system = copy.copy(def_system)
b = get_b(system, n)
A = get_a(system, n)

print('исходная система:')
print(system, '\n')

forward(system, n, 1)
x_g = get_solution(system, n)

print('результат методом Гаусса:')
for i in range (0, n):
  print('x' + "{}".format(i) + ': ', x_g[i])

исходная система:
[[ 3.278164  1.046583 -1.378574 -0.527466]
 [ 1.046583  2.975937  0.934251  2.526877]
 [-1.378574  0.934251  4.836173  5.165441]] 

результат методом Гаусса:
x0:  0.10000019313508599
x1:  0.4999996300255529
x2:  1.0000001058481585


**Приведение системы к виду x = Hx + g**

In [ ]:
#вычисление inf нормы матрицы
def get_norm_matrix(matrix, n):
  norm = 0
  for i in range(0, n):
    for j in range(0, n):
      if abs(matrix[i, j]) > norm:
        norm = ma
  return norm

#вычисление inf нормы вектора
def get_norm_vector(x, n):
  norm = 0
  for i in range(0, n):
    if abs(x[i]) > norm:
      norm = abs(x[i])
  return norm

system = copy.copy(def_system)
b = get_b(system, n)
A = get_a(system, n)

H = np.zeros((n, n))
for i in range(0, n):
  for j in range(0, n):
    if i != j:
      H[i, j] = (-A[i, j]) / A[i, i]

g = np.zeros(n)
for i in range(0, n):
  g[i] = b[i] / A[i, i]

print('исходная система:')
print(system, '\n')

print('H:\n', H, '\n')
print('g:\n', g, '\n')


print('inf норма H: ', get_norm_matrix(H, n))
print('спектральный радиус H: ', spec_rad)

исходная система:
[[ 3.278164  1.046583 -1.378574 -0.527466]
 [ 1.046583  2.975937  0.934251  2.526877]
 [-1.378574  0.934251  4.836173  5.165441]] 

H:
 [[ 0.         -0.31925889  0.42053235]
 [-0.35168184  0.         -0.31393507]
 [ 0.28505473 -0.19317981  0.        ]] 

g:
 [-0.16090287  0.84910299  1.06808441] 

inf норма H:  0.7397912368020636
спектральный радиус H:  0.620403


**Метод простой итерации**

In [ ]:
num_app = 7

approximations = []
approximations.append(np.zeros(n))

for k in range(1, num_app + 1):
  x = np.zeros(n)
  for i in range(0, n):
    sum = 0
    for j in range(0, n):
      sum += H[i, j] * (approximations[k - 1])[j]
    x[i] = sum + g[i]
  approximations.append(x)

x_iter = approximations[num_app]

norm_g = get_norm_vector(g, n)
norm_H = get_norm_matrix(H, n)

apriori_err = pow(norm_H, num_app) * get_norm_vector(approximations[0], n) + pow(norm_H, num_app) / (1 - norm_H) * norm_g

error_vector = np.zeros(n)
for i in range(0, n):
  error_vector[i] = x_g[i] - x_iter[i]
actual_err = get_norm_vector(error_vector, n)

error_vector = np.zeros(n)
for i in range(0, n):
  error_vector[i] = (approximations[num_app])[i] - (approximations[num_app - 1])[i]
error = get_norm_vector(error_vector, n)
posterior_err = norm_H / (1 - norm_H) * error

print('решение по Гауссу:\n', x_g, '\n')
print('приближение решения:')
for i in range (0, num_app + 1):
  print(i, ' --- ', approximations[i])

print('\nфактическая погрешность: ', actual_err)
print('\nапостериорная погрешность: ', posterior_err)
print('\nаприорная погрешность: ', apriori_err)

x_luster = np.zeros(n)
for i in range(0, n):
  x_luster[i] = (approximations[num_app - 1])[i] + 1 / (1 - spec_rad) * ((approximations[num_app])[i] - (approximations[num_app - 1])[i])
print('\nуточнение решения по Люстернику', x_luster)

error_vector = np.zeros(n)
for i in range(0, n):
  error_vector[i] = x_g[i] - x_luster[i]
actual_err_l = get_norm_vector(error_vector, n)

print('погрешность: ', actual_err_l)


решение по Гауссу:
 [0.10000019 0.49999963 1.00000011] 

приближение решения:
0  ---  [0. 0. 0.]
1  ---  [-0.16090287  0.84910299  1.06808441]
2  ---  [0.0171775  0.57038045 0.85818873]
3  ---  [0.01789422 0.57364643 0.96279495]
4  ---  [0.06084183 0.54055481 0.96236833]
5  ---  [0.07122721 0.52558485 0.98100339]
6  ---  [0.08384315 0.5160823  0.98685568]
7  ---  [0.089338   0.50980826 0.99228762]

фактическая погрешность:  0.010662190445432818

апостериорная погрешность:  0.017837514946761987

априорная погрешность:  0.49779202757005997

уточнение решения по Люстернику [0.09831864 0.49955415 1.00116542]
погрешность:  0.00168155081285018


**Метод Зейделя** 

In [ ]:
approximations = []
approximations.append(np.zeros(n))

for k in range(1, num_app + 1):
  x = np.zeros(n)
  for i in range(0, n):
    sum = 0
    for j in range(0, i):
      sum += H[i, j] * x[j]
    for j in range(i, n):
      sum += H[i, j] * (approximations[k - 1])[j]
    x[i] = sum + g[i]
  approximations.append(x)

error_vector = np.zeros(n)
for i in range(0, n):
  error_vector[i] = x_g[i] - (approximations[num_app])[i]
actual_err_z = get_norm_vector(error_vector, n)

print('решение по Гауссу:\n', x_g, '\n')

print('решение методом простой итерации:\n', x_iter)
print('погрешность: ', actual_err, '\n')

x_z = approximations[num_app]
print('решение методом Зейделя:\n', x_z)
print('погрешность: ', actual_err_z)

print('\nприближение решения методом Зейделя:')
for i in range (0, num_app + 1):
  print(i, ' --- ', approximations[i])

решение по Гауссу:
 [0.10000019 0.49999963 1.00000011] 

решение методом простой итерации:
 [0.089338   0.50980826 0.99228762]
погрешность:  0.010662190445432818 

решение методом Зейделя:
 [0.0982971  0.50116494 0.99928952]
погрешность:  0.001703096258481418

приближение решения методом Зейделя:
0  ---  [0. 0. 0.]
1  ---  [-0.16090287  0.90568961  0.84725734]
2  ---  [-0.09375321  0.6160905   0.92234337]
3  ---  [0.03027998 0.54889814 0.97067983]
4  ---  [0.07205878 0.51903078 0.98835884]
5  ---  [0.0890288  0.50751267 0.99542129]
6  ---  [0.09567605 0.5029578  0.99819603]
7  ---  [0.0982971  0.50116494 0.99928952]


**Метод верхней релаксации**

In [ ]:
q = 2 / (1 + pow(1 - spec_rad * spec_rad, 0.5))

approximations = []
approximations.append(np.zeros(n))

for k in range(1, num_app + 1):
  x = np.zeros(n)
  for i in range(0, n):
    sum = 0
    for j in range(0, i):
      sum += H[i, j] * x[j]
    for j in range(i + 1, n):
      sum += H[i, j] * (approximations[k - 1])[j]
    x[i] = (approximations[k - 1])[i] + q * (sum + g[i] - (approximations[k - 1])[i])
  approximations.append(x)

error_vector = np.zeros(n)
for i in range(0, n):
  error_vector[i] = x_g[i] - (approximations[num_app])[i]
actual_err_r = get_norm_vector(error_vector, n)

print('решение по Гауссу:\n', x_g, '\n')

print('решение методом простой итерации:\n', x_iter)
print('погрешность: ', actual_err, '\n')

print('решение методом Зейделя:\n', x_z)
print('погрешность: ', actual_err_z)

print('\nрешение методом верхней релаксации:\n', approximations[num_app])
print('погрешность: ', actual_err_r)

print('\nприближение решения методом верхней релаксации:')
for i in range (0, num_app + 1):
  print(i, ' --- ', approximations[i])

решение по Гауссу:
 [0.10000019 0.49999963 1.00000011] 

решение методом простой итерации:
 [0.089338   0.50980826 0.99228762]
погрешность:  0.010662190445432818 

решение методом Зейделя:
 [0.0982971  0.50116494 0.99928952]
погрешность:  0.001703096258481418

решение методом верхней релаксации:
 [0.09996447 0.50004066 0.99999232]
погрешность:  4.1026680938860416e-05

приближение решения методом верхней релаксации:
0  ---  [0. 0. 0.]
1  ---  [-0.18035576  1.02285426  0.91810354]
2  ---  [-0.09181622  0.54121986  0.93968697]
3  ---  [0.08000949 0.52412005 0.99568157]
4  ---  [0.09174974 0.50185548 0.99748419]
5  ---  [0.09914759 0.50099668 0.99981596]
6  ---  [0.09965967 0.50007812 0.99989657]
7  ---  [0.09996447 0.50004066 0.99999232]
